<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3%E3%82%B5%E3%83%BC%E3%83%81%E7%B0%A1%E6%98%93Web_API%E3%81%8C%E6%8F%90%E4%BE%9B%E3%81%99%E3%82%8B%E9%A1%9E%E4%BC%BC%E7%94%BB%E5%83%8F%E6%A4%9C%E7%B4%A2%E3%81%AE%E7%B5%90%E6%9E%9C%E3%82%92%E3%82%AE%E3%83%A3%E3%83%A9%E3%83%AA%E3%83%BC%E3%81%A7%E8%A1%A8%E7%A4%BA%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ジャパンサーチ簡易Web APIが提供する類似画像検索の結果をギャラリーで表示する

In [ ]:
!pip install -U jps_gallery_tools

In [ ]:
# id = "cobas-58264"
id = "FishPix-KPM_NR_54762"

In [ ]:
import requests
import json
url = f"https://jpsearch.go.jp/api/item/search/jps-cross?image={id}"
df = requests.get(url).json()

In [ ]:
from jps_gallery_tools.api import *
gallery = JpsCuration()

In [ ]:
gallery.setLangMetadata('title', 'ja', 'ギャラリーの自動生成')
gallery.setLangMetadata('summary', 'ja', 'Google Colabを用いて、ギャラリーの自動生成を行います。')
gallery.setLangMetadata("responsibility", "ja", "中村覚")

## テキスト

In [ ]:
part_01= JpsCurationText()
gallery.addPart(part_01)
part_01.addText("ja", "ジャパンサーチ簡易Web APIが提供する類似画像検索の結果をギャラリーで表示する")

## リスト

In [ ]:
part_02 = JpsCurationList("tile")
part_02.setLangMetadata("title", "ja", "検索結果の一覧")
gallery.addPart(part_02)

画像を追加

In [ ]:
items = []
years = []

for item_r in df["list"]:
    item = JpsCurationListExternalLink()
    items.append(item)
    

    common = item_r["common"]
    title = common["title"]

    provider = common["provider"]

    item.setLangMetadata("title", "ja", title)

    thumbnail_url = common["thumbnailUrl"][0] if "thumbnailUrl" in common else ""
    original_url = common["contentsUrl"][0] if "contentsUrl" in common else thumbnail_url
    source = provider
    top_image = JpsCurationUrlImage(original_url, title, source, thumbnail_url)
    
    item.setImage(top_image)

    if "coordinates" in common:
      coordinates = common["coordinates"]
      item.setMetadata("latlon", {
        "lat": coordinates["lat"],
        "lon": coordinates["lon"],
      })

    if "temporal" in common:
      temporals = common["temporal"]
      for temporal in temporals:
        try:
          year = int(temporal[0:4])
          if year not in years:
            years.append(year)
        except:
          pass

for item in items:
  part_02.addPart(item)

In [ ]:
items[0].export()

## 年表

In [ ]:
part_03 = JpsCurationTimeline(1800, 2050)
gallery.addPart(part_03)
part_03.setLangMetadata("title", "ja", "年表")


In [ ]:
years.sort()

for i in range(len(years)):
  year = years[i]
  part_03_01 = JpsCurationTimepoint(year, year, year, year, labelHeight=i+1)
  part_03.addPart(part_03_01)

## 地図

In [ ]:
part_04 = JpsCurationList("map")
gallery.addPart(part_04)
part_04.setLangMetadata("title", "ja", "地図表示")

part_04.setMetadata("openActionType", "link")
part_04.setMetadata("mapParam", {
    "center": {
        "lat": 35.713736762125464,
        "lon": 139.76272050237696
    },
    "zoom": 5,
    "mode": "NORMAL"
})


地図にアイテムの追加

In [ ]:
import copy

for item in items:
  item_04_01 = copy.copy(item)
  part_04.addPart(item_04_01)

## 出力

In [ ]:
b = gallery.encode(curationType="GALLERY")

In [ ]:
import IPython
from IPython.display import display, HTML
 
htm = HTML(f'''\
<div
        class="jps"
        data-lang="ja"
        data-cur="{b}">
</div>
<script src="https://jpsearch.go.jp/assets/js/wp.bundle.js"></script>
''')
 
display(htm)